# Example 2: Coupled Transmon Qubits

In [ ]:
%load_ext autoreload
%autoreload 2
import time

In [ ]:
import design as d
import json
with open('design_variables.json') as in_file:
    dv = json.load(in_file)


import qdesignoptimizer.utils.utils_design_variables as u
from qdesignoptimizer.utils.utils_design import create_chip_base
from qdesignoptimizer.utils.utils import close_ansys

## Design assembly

In [ ]:
CHIP_NAME = "multi_transmon_chip"
OPEN_GUI = True
CHIP_TYPE = {"size_x": "10mm",
             "size_y": "10mm",
             "size_z": "-300um"}
design, gui = create_chip_base(chip_name=CHIP_NAME, chip_type=CHIP_TYPE, open_gui=OPEN_GUI)

u.add_design_variables_to_design(design, dv)

In [ ]:
def render_qiskit_metal_design(design, gui):
    d.add_branch(design, 0)
    d.add_branch(design, 2)

    d.add_resonator_coupler(design, 5, 0, 2)
 
    d.add_route_interconnects(design, 0,2, 'start_end')
 
    d.add_launch_pads(design)

    d.add_chargeline(design, 0)
    d.add_chargeline(design, 2)

    gui.rebuild()
    gui.autoscale()

render_qiskit_metal_design(design, gui)


## MiniStudies

In [ ]:
import mini_studies as ms
import optimization_targets as ot
import target_parameters as tp
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState

### Single resonator-qubit system

In [ ]:
MINI_STUDY_BRANCH = 0
MINI_STUDY = ms.get_mini_study_qb_res(branch=MINI_STUDY_BRANCH)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = []
for target in ot.get_opt_targets_qb_res(
        branch=MINI_STUDY_BRANCH,
    ):
    opt_targets.append(target)

In [ ]:
design_analysis_state = DesignAnalysisState(design, RENDER_QISKIT_METAL, tp.TARGET_PARAMS)
design_analysis = DesignAnalysis(
    design_analysis_state, 
    mini_study=MINI_STUDY, 
    opt_targets=opt_targets,
    save_path=CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_parameters= False,
    plot_settings=ps.PLOT_SETTINGS,
    plot_branches_separately=False
    )

nbr_runs = 2
nbr_passes = 2
delta_f = 0.001
for i in range(nbr_runs):   
    design_analysis.update_nbr_passes(nbr_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run= i)

In [ ]:
design_analysis.overwrite_parameters()

### Two qubit-resonator system

In [ ]:
MINI_STUDY_BRANCH = [2,0]
MINI_STUDY = ms.get_mini_study_2qb_resonator_coupler(branches=MINI_STUDY_BRANCH, coupler=5)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = []
for target in ot.get_opt_targets_2qb_resonator_coupler(
        branches=MINI_STUDY_BRANCH,
    ):
    opt_targets.append(target)

In [ ]:
design_analysis_state = DesignAnalysisState(design, RENDER_QISKIT_METAL, tp.TARGET_PARAMS)
design_analysis = DesignAnalysis(
    design_analysis_state, 
    mini_study=MINI_STUDY, 
    opt_targets=opt_targets,
    save_path=CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_parameters= False,
    plot_settings=ps.PLOT_SETTINGS,
    plot_branches_separately=False
    )

nbr_runs = 2
nbr_passes = 2
delta_f = 0.001
for i in range(nbr_runs):   
    design_analysis.update_nbr_passes(nbr_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run= i)

In [ ]:
design_analysis.overwrite_parameters()

## View Optimization results

In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

In [ ]:
close_ansys()